#### Importar librerias


In [ ]:
# Manipuladcion de datos 
import pandas as pd 
import numpy as np

#libreria para descargar datos de productos financieros
import yfinance as yf 

# Calculo de indicadores tecnicos 'ta package' 
import ta as ta  
from ta.momentum import RSIIndicator, StochasticOscillator,AwesomeOscillatorIndicator
from ta.trend import MACD, ADXIndicator,IchimokuIndicator
from ta.volatility import BollingerBands, AverageTrueRange
from ta.trend import SMAIndicator, EMAIndicator


[enlace a Documentacion libreria TA]: https://technical-analysis-library-in-python.readthedocs.io/en/latest/ta.html
[**Visita la documentación libreria TA** ][enlace a Documentacion libreria TA]

#### Extraccion de datos OHLCV (Open,High,Low,Close,Volume) del ticket definido 
- Elegimos Tickets
- Funcion extract_clean_ticker() para crear el dataset inicial y tener los datos historicos. 

In [ ]:
# Definimos los tickets 
vandguard_sp500 = 'VOO'


In [3]:
def extract_clean_ticker(ticker: str, start=None, end=None, interval='1d') -> pd.DataFrame:
    """
    Descarga y limpia datos históricos de un ticker con yfinance.
    
    Args:
        ticker (str): Símbolo del ticker (ej: 'VOO', '^GSPC')
        start (str): Fecha de inicio 'YYYY-MM-DD'
        end (str): Fecha de fin 'YYYY-MM-DD'
        interval (str): Intervalo temporal ('1d', '1wk', etc.)
        
    Returns:
        pd.DataFrame: DataFrame limpio con columnas estandarizadas.
    """
    df = yf.download(
        tickers=ticker,
        start=start,
        end=end,
        interval=interval,
        auto_adjust=True,
        progress=False
    )
    

    if isinstance(df.columns, pd.MultiIndex):             # Si las columnas del DataFrame son un MultiIndex,
        df.columns = df.columns.droplevel(1)              # se elimina el segundo nivel del MultiIndex para simplificar.
        df.columns = df.columns.get_level_values(0)       # se selecciona solo el primer nivel de nombres de columna para simplificar el DataFrame.
    
    df = df.reset_index()
    df.rename(columns={"Date": "date"}, inplace=True)
    df['date'] = pd.to_datetime(df['date'])
    df.columns = [col.lower() for col in df.columns]
    df = df.sort_values('date')                           # datos ordenados por fecha
    df = df.round(3)                                      # Redondear los valores float a 3 decimales
    
    # Asegurarse de que las columnas necesarias están presentes
    cols_esperadas = {'open', 'high', 'low', 'close', 'volume'}
    if not cols_esperadas.issubset(df.columns):
        raise ValueError(f"Faltan columnas necesarias en {ticker}: {cols_esperadas - set(df.columns)}")
    
    return df

In [4]:
#extraemos los datos para VOO
df_voo = extract_clean_ticker(vandguard_sp500, start='2010-01-01', end='2025-06-01')
#cambiamos la columna date a tipo datetime
df_voo['date'] = pd.to_datetime(df_voo['date'])


#### Agregar indicadores tecnicos al dataframe

In [ ]:

def añadir_indicadores_tecnicos(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calcula e incorpora indicadores técnicos clásicos y divergencias basadas en volumen
    al DataFrame de precios OHLCV para análisis diario.
    
    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame con columnas 'open', 'high', 'low', 'close', 'volume'.
    
    Retorna:
    --------
    pd.DataFrame
        DataFrame con columnas adicionales para cada indicador y divergencias.
    
    Indicadores incluidos:
    ---------------------
    - RSI (Relative Strength Index) : fuerza relativa de precios.
    - MACD : tendencia y momentum; columnas macd, macd_signal, macd_diff.
    - Estocástico (%K, %D) : sobrecompra/sobreventa.
    - Bandas de Bollinger : volatilidad y niveles de soporte/resistencia dinámicos.
    - ATR (Average True Range) : volatilidad real del mercado.
    - SMA y EMA : medias móviles simples y exponenciales.
    - ADX : fuerza de tendencia.
    - Divergencias de RSI y MACD filtradas por volumen.
    """
    df = df.copy()

    # RSI
    rsi = RSIIndicator(close=df['close'], window=14)
    df['rsi'] = rsi.rsi()  # Oscilador de fuerza relativa

    # MACD
    macd = MACD(close=df['close'])
    df['macd'] = macd.macd()  # Línea MACD
    df['macd_signal'] = macd.macd_signal()  # Línea de señal
    df['macd_diff'] = macd.macd_diff()  # Histograma MACD

    # Estocástico
    stoch = StochasticOscillator(high=df['high'], low=df['low'], close=df['close'], window=14, smooth_window=3)
    df['stochastic_k'] = stoch.stoch()  # Línea %K
    df['stochastic_d'] = stoch.stoch_signal()  # Línea %D

    # Bandas de Bollinger
    bb = BollingerBands(close=df['close'], window=20, window_dev=2)
    df['bollinger_hband'] = bb.bollinger_hband()  # Banda superior
    df['bollinger_lband'] = bb.bollinger_lband()  # Banda inferior
    df['bollinger_mavg'] = bb.bollinger_mavg()    # Media central
    df['bollinger_width'] = bb.bollinger_wband()  # Ancho de banda : valores entre hband - lband

    # ATR
    atr = AverageTrueRange(high=df['high'], low=df['low'], close=df['close'], window=14)
    df['atr'] = atr.average_true_range()  # Medida de volatilidad

    # SMA y EMA
    df['sma_20'] = SMAIndicator(close=df['close'], window=20).sma_indicator()
    df['sma_50'] = SMAIndicator(close=df['close'], window=50).sma_indicator()
    df['sma_200'] = SMAIndicator(close=df['close'], window=200).sma_indicator()
    df['ema_20'] = EMAIndicator(close=df['close'], window=20).ema_indicator()
    df['ema_50'] = EMAIndicator(close=df['close'], window=50).ema_indicator()
    df['ema_200'] = EMAIndicator(close=df['close'], window=200).ema_indicator()
    
    # Ichimoku Cloud: identifica soporte, resistencia y tendencia
    ichimoku = IchimokuIndicator(high=df['high'], low=df['low'], window1=9, window2=26, window3=52)
    df['ichimoku_a'] = ichimoku.ichimoku_a()
    df['ichimoku_b'] = ichimoku.ichimoku_b()
    df['ichimoku_base'] = ichimoku.ichimoku_base_line()
    df['ichimoku_conversion'] = ichimoku.ichimoku_conversion_line()

    # ADX
    adx = ADXIndicator(high=df['high'], low=df['low'], close=df['close'], window=14)
    df['adx'] = adx.adx()  # Fuerza de tendencia

    # Volumen relativo
    df['volume_sma_20'] = SMAIndicator(close=df['volume'], window=20).sma_indicator()
    df['volume_ratio'] = df['volume'] / df['volume_sma_20']  # Para usar en filtrado de divergencias
    
        # Awesome Oscillator: mide la presión del mercado y puede usarse para divergencias con volumen
    ao = AwesomeOscillatorIndicator(high=df['high'], low=df['low'])
    df['awesome_osc'] = ao.awesome_oscillator()
    
    # Redondear todas las columnas numéricas a 3 decimales
    df = df.round(3)
    # Eliminar filas con valores nulos por cálculos
    df = df.dropna().reset_index(drop=True)

    return df


In [6]:
df_voo = añadir_indicadores_tecnicos(df_voo)

In [7]:
#exportamos los datos a un csv en la carpeta data
df_voo.to_csv('../data/VOO.csv', index=False)

In [8]:
df_voo.head()

,date,close,high,low,open,volume,rsi,macd,macd_signal,macd_diff,...,ema_50,ema_200,ichimoku_a,ichimoku_b,ichimoku_base,ichimoku_conversion,adx,volume_sma_20,volume_ratio,awesome_osc
0,2011-06-23,91.127,91.158,89.689,90.338,260750,44.615,-0.773,-0.887,0.114,...,92.586,89.286,91.583,93.16,92.395,90.771,21.806,116412.5,2.240,-2.011
1,2011-06-24,90.125,91.181,89.969,91.150,114500,39.854,-0.802,-0.870,0.067,...,92.489,89.295,91.583,93.16,92.395,90.771,21.981,119115.0,0.961,-1.842
2,2011-06-27,90.886,91.228,89.985,90.187,77850,44.682,-0.756,-0.847,0.091,...,92.426,89.310,91.564,93.16,92.356,90.771,22.092,120627.5,0.645,-1.670
3,2011-06-28,92.051,92.082,91.181,91.274,59500,51.147,-0.618,-0.801,0.184,...,92.412,89.338,91.564,93.16,92.356,90.771,21.284,114977.5,0.517,-1.544
4,2011-06-29,92.875,92.999,92.191,92.533,125350,55.140,-0.436,-0.728,0.292,...,92.430,89.373,91.850,93.16,92.356,91.344,19.838,110145.0,1.138,-1.305


#### Señales Técnicas Diarias Incluidas en el Proyecto

Este proyecto genera señales de trading basadas en indicadores técnicos diarios y divergencias. A continuación se detallan las señales disponibles:

##### 1. SMA/EMA (Medias Móviles)
- **Golden Cross**: SMA/EMA 50 cruza sobre SMA/EMA 200 → señal de compra.
- **Death Cross**: SMA/EMA 50 cruza bajo SMA/EMA 200 → señal de venta.
- **Cruce corto/medio**: EMA/SMA 20 cruza EMA/SMA 50 → señal de compra/venta según la dirección del cruce.

##### 2. Ichimoku Cloud
- **Precio sobre la nube** → tendencia alcista.
- **Precio bajo la nube** → tendencia bajista.
- **Cruce Tenkan/Kijun** → compra/venta según la dirección del cruce.
- **Cruce Chikou Span** → confirmación de tendencia.

##### 3. ADX (Average Directional Index)
- **ADX > 25** → tendencia fuerte detectada.
- **+DI cruza -DI** → señal de compra.
- **-DI cruza +DI** → señal de venta.

##### 4. RSI (Relative Strength Index)
- **RSI > 70** → sobrecompra, posible reversión a la baja.
- **RSI < 30** → sobreventa, posible reversión al alza.
- **Divergencia alcista** → precio baja pero RSI sube → señal de compra.
- **Divergencia bajista** → precio sube pero RSI baja → señal de venta.

##### 5. MACD (Moving Average Convergence Divergence)
- **MACD cruza señal hacia arriba** → señal de compra.
- **MACD cruza señal hacia abajo** → señal de venta.
- **Divergencia alcista** → precio baja mientras MACD sube → señal de compra.
- **Divergencia bajista** → precio sube mientras MACD baja → señal de venta.

##### 6. Estocástico
- **%K/%D cruza hacia arriba en zona de sobreventa (<20)** → señal de compra.
- **%K/%D cruza hacia abajo en zona de sobrecompra (>80)** → señal de venta.
- **Precio sobrecompra (>80)** → posible reversión a la baja.
- **Precio sobreventa (<20)** → posible reversión al alza.
- **Divergencia alcista** → precio baja mientras estocástico sube → señal de compra.
- **Divergencia bajista** → precio sube mientras estocástico baja → señal de venta.

##### 7. Bollinger Bands
- **Precio rompe banda superior** → señal de venta potencial.
- **Precio rompe banda inferior** → señal de compra potencial.
- **Reversión hacia banda media** → posible ajuste de tendencia.

##### 8. ATR (Average True Range)
- **Cambio abrupto en volatilidad** → señal de precaución, ajustar stops o tamaño de posición.

---